# Object systematics
This is a rendered copy of [systematics.ipynb](https://github.com/scikit-hep/coffea/blob/master/binder/systematics.ipynb). You can optionally run it interactively on [binder at this link](https://mybinder.org/v2/gh/coffeateam/coffea/master?filepath=binder%2Fsystematics.ipynb)

This notebook presents how to add systematics to objects in coffea.\
Coffea currently implements two types of object systematics. `UpDownSystematic` which varies one field on the object it is applied on and `UpDownMultiSystematic` which varies multiple fields on the object it is applied on.\
Check the snippets below for example usage.

In [1]:
import awkward
import numpy as np
from coffea import nanoevents


def get_array(array):
    return array.compute() if nanoevents_mode == "dask" else array


nanoevents_mode = "virtual"

access_log = []
events = nanoevents.NanoEventsFactory.from_root({"coffea/tests/samples/nano_dy.root": "Events"}, mode=nanoevents_mode, access_log=access_log).events()

muons = events.Muon
jets = events.Jet
met = events.MET

/home/iason/Dropbox/work/pyhep_dev/coffea/src/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/home/iason/Dropbox/work/pyhep_dev/coffea/src/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_genPartIdx => GenPart
  warnings.warn(
/home/iason/Dropbox/work/pyhep_dev/coffea/src/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(
/home/iason/Dropbox/work/pyhep_dev/coffea/src/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for FatJet_genJetAK8Idx => GenJetAK8
  warnings.warn(


In [2]:
def some_event_weight(ones):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ones[:, None]


events.add_systematic("RenFactScale", "UpDownSystematic", "weight", some_event_weight)
events.add_systematic("XSectionUncertainty", "UpDownSystematic", "weight", some_event_weight)


def muon_pt_scale(pt):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * pt[:, None]


def muon_pt_resolution(pt):
    return np.random.normal(pt[:, None], np.array([0.02, 0.01], dtype=np.float32))


def muon_eff_weight(ones):
    return (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ones[:, None]


def muon_pt_phi_systematic(ptphi):
    pt_var = (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ptphi.pt[:, None]
    phi_var = (1.0 + np.array([0.1, -0.1], dtype=np.float32)) * ptphi.phi[:, None]
    return awkward.zip({"pt": pt_var, "phi": phi_var}, depth_limit=1)


muons.add_systematic("PtScale", "UpDownSystematic", "pt", muon_pt_scale)
muons.add_systematic("PtResolution", "UpDownSystematic", "pt", muon_pt_resolution)
muons.add_systematic("EfficiencySF", "UpDownSystematic", "weight", muon_eff_weight)
muons.add_systematic("PtPhiSystematic", "UpDownMultiSystematic", ("pt", "phi"), muon_pt_phi_systematic)


def jet_pt_scale(pt):
    return (1.0 + np.array([0.10, -0.10], dtype=np.float32)) * pt[:, None]


def jet_pt_resolution(pt):
    return np.random.normal(pt[:, None], np.array([0.20, 0.10], dtype=np.float32))


def jet_pt_phi_systematic(ptphi):
    pt_var = (1.0 + np.array([0.10, -0.10], dtype=np.float32)) * ptphi.pt[:, None]
    phi_var = (1.0 + np.array([0.2, -0.2], dtype=np.float32)) * ptphi.phi[:, None]
    return awkward.zip({"pt": pt_var, "phi": phi_var}, depth_limit=1)


jets.add_systematic("PtScale", "UpDownSystematic", "pt", jet_pt_scale)
jets.add_systematic("PtResolution", "UpDownSystematic", "pt", jet_pt_resolution)
jets.add_systematic("PtPhiSystematic", "UpDownMultiSystematic", ("pt", "phi"), jet_pt_phi_systematic)


def met_pt_scale(pt):
    return (1.0 + np.array([0.03, -0.03], dtype=np.float32)) * pt[:, None]


def met_pt_phi_systematic(ptphi):
    pt_var = (1.0 + np.array([0.03, -0.03], dtype=np.float32)) * ptphi.pt[:, None]
    phi_var = (1.0 + np.array([0.05, -0.05], dtype=np.float32)) * ptphi.phi[:, None]
    return awkward.zip({"pt": pt_var, "phi": phi_var}, depth_limit=1)


met.add_systematic("PtScale", "UpDownMultiSystematic", "pt", met_pt_scale)
met.add_systematic("PtPhiSystematic", "UpDownMultiSystematic", ("pt", "phi"), met_pt_phi_systematic)

In [3]:
renfact_up = events.systematics.RenFactScale.up.weight_RenFactScale
get_array(renfact_up)

<Array [1.05, 1.05, 1.05, 1.05, ..., 1.05, 1.05, 1.05] type='40 * float64'>

In [4]:
muon_pt = awkward.flatten(muons.pt)
get_array(muon_pt)

<Array [76.8, 20.1, 31, ..., 4.36, 10.1, 17.9] type='18 * float32[parameter...'>

In [5]:
muon_PtScale_up = awkward.flatten(muons.systematics.PtScale.up)
get_array(muon_PtScale_up)

<MuonArray [Muon, ..., Muon] type='18 * Muon[dxy: float32[parameters={"__do...'>

In [6]:
muon_PtScale_up_pt = awkward.flatten(muons.systematics.PtScale.up.pt)
get_array(muon_PtScale_up_pt)

<Array [80.6, 21.1, 32.6, 53.2, ..., 4.58, 10.6, 18.8] type='18 * float32'>

In [7]:
muons_PtPhiSystematic_up = awkward.flatten(muons.systematics.PtPhiSystematic.up)
get_array(muons_PtPhiSystematic_up)

<MuonArray [Muon, ..., Muon] type='18 * Muon[dxy: float32[parameters={"__do...'>

In [8]:
muons_PtPhiSystematic_up_pt = awkward.flatten(muons.systematics.PtPhiSystematic.up.pt)
get_array(muons_PtPhiSystematic_up_pt)

<Array [80.6, 21.1, 32.6, 53.2, ..., 4.58, 10.6, 18.8] type='18 * float32'>

In [9]:
muons_PtPhiSystematic_up_phi = awkward.flatten(muons.systematics.PtPhiSystematic.up.phi)
get_array(muons_PtPhiSystematic_up_phi)

<Array [1.8, -3.16, -1.54, 0.522, ..., -0.784, -2.41, 3.19] type='18 * float32'>

In [10]:
jets_pt = awkward.flatten(jets.pt)
get_array(jets_pt)

<Array [80.8, 45.6, 29.6, ..., 18.8, 18.3] type='188 * float32[parameters={...'>

In [11]:
jets_PtScale_up = awkward.flatten(jets.systematics.PtScale.up)
get_array(jets_PtScale_up)

<JetArray [Jet, ...] type='188 * Jet[area: float32[parameters={"__doc__": "...'>

In [12]:
jets_PtScale_up_pt = awkward.flatten(jets.systematics.PtScale.up.pt)
get_array(jets_PtScale_up_pt)

<Array [88.8, 50.2, 32.6, 19.3, ..., 16.7, 20.7, 20.1] type='188 * float32'>

In [13]:
jets_PtPhiSystematic_up = awkward.flatten(jets.systematics.PtPhiSystematic.up)
get_array(jets_PtPhiSystematic_up)

<JetArray [Jet, ...] type='188 * Jet[area: float32[parameters={"__doc__": "...'>

In [14]:
jets_PtPhiSystematic_up_pt = awkward.flatten(jets.systematics.PtPhiSystematic.up.pt)
get_array(jets_PtPhiSystematic_up_pt)

<Array [88.8, 50.2, 32.6, 19.3, ..., 16.7, 20.7, 20.1] type='188 * float32'>

In [15]:
jets_PtPhiSystematic_up_phi = awkward.flatten(jets.systematics.PtPhiSystematic.up.phi)
get_array(jets_PtPhiSystematic_up_phi)

<Array [-1.09, 1.98, -3.43, -2.77, ..., 1.89, 3.14, 2.85] type='188 * float32'>

In [16]:
met_pt = met.pt
get_array(met_pt)

<Array [25.9, 40.5, 38.7, ..., 24.2, 42.5] type='40 * float32[parameters={"...'>

In [17]:
met_PtScale_up = met.systematics.PtScale.up
get_array(met_PtScale_up)

<MissingETArray [MissingET, ...] type='40 * MissingET[MetUnclustEnUpDeltaX:...'>

In [18]:
met_PtScale_up_pt = met.systematics.PtScale.up.pt
get_array(met_PtScale_up_pt)

<Array [26.7, 41.7, 39.9, 9.69, ..., 25.6, 24.9, 43.8] type='40 * float32'>

In [19]:
met_PtPhiSystematic_up = met.systematics.PtPhiSystematic.up
get_array(met_PtPhiSystematic_up)

<MissingETArray [MissingET, ...] type='40 * MissingET[MetUnclustEnUpDeltaX:...'>

In [20]:
met_PtPhiSystematic_up_pt = met.systematics.PtPhiSystematic.up.pt
get_array(met_PtPhiSystematic_up_pt)

<Array [26.7, 41.7, 39.9, 9.69, ..., 25.6, 24.9, 43.8] type='40 * float32'>

In [21]:
met_PtPhiSystematic_up_phi = met.systematics.PtPhiSystematic.up.phi
get_array(met_PtPhiSystematic_up_phi)

<Array [1.75, 2.05, -2.47, 3.01, ..., -2.47, -2.54, -0.687] type='40 * float32'>

In [ ]:
# TODO: Make it so that syst_muons.Y > X returns boolean values
#       for all variations over Y.
#       Requires some tracking of (pieces of) "what".